# **Stock Price Prediction using LSTM and RNN**

## Project Overview

Stock price prediction is a valuable tool in financial markets, benefiting businesses, investors, and financial institutions.

- For investors and fund managers, accurate stock price predictions enable better decision-making. These predictions help in identifying potential profit opportunities and minimizing losses, allowing for smarter allocation of capital and portfolio adjustments based on expected price changes.

- Machine learning and deep learning techniques have shown promise in improving stock price predictions, providing useful insights for businesses. These methods can refine investment strategies, offer a competitive edge, and enhance risk management in the complex stock market environment.

- Predicting stock prices is challenging due to market volatility and complexity. Traditional methods often struggle to capture subtle patterns in stock price data. However, recurrent neural networks (RNNs) and long short-term memory (LSTM) models are capable of revealing temporal relationships and making accurate predictions in various time series forecasting tasks.

- Throughout the project, we will explore the basics of RNNs and LSTMs and their ability to capture temporal patterns in Tesla stock price data. We will also cover data preparation steps to get historical stock price data ready for training and testing our models.

## **Outcomes**

* Load and preprocess time series data for Tesla stock prices, ensuring data quality and consistency.

* Approach Tesla stock price prediction as a time series forecasting problem, respecting the temporal order of data and creating distinct training and testing sets.

* Recurrent neural networks (RNNs) and their capacity to handle sequential data and capture temporal dependencies.

* Long short-term memory (LSTM) networks, a specialized variant of RNNs optimized for capturing long-term dependencies.

* Incorporation of additional features or factors, such as Relative Strength Index (RSI) and Exponential Moving Average (EMA), to create a multivariate input model that enhances prediction accuracy.

* Recognize the challenges and constraints inherent in Tesla stock price prediction, including market volatility, unforeseen events, and the presence of noise in financial markets.

## Key Libraries Used

* **yfinance**: yfinance simplifies the process of downloading historical market data from Yahoo Finance. [yfinance GitHub repository](https://github.com/ranaroussi/yfinance).

* **pandas_datareader**: pandas_datareader provides convenient access to online data sources, including Yahoo Finance, for retrieving financial data. [pandas_datareader documentation](https://pandas-datareader.readthedocs.io/)

* **pandas_ta**: pandas_ta expands Pandas capabilities by introducing technical analysis indicators for analyzing financial data. [pandas_ta GitHub repository](https://github.com/twopirllc/pandas-ta).


### Install Packages


In [ ]:
# Install required packages
import warnings
warnings.filterwarnings('ignore')

# Updated package versions for better compatibility and performance
!pip install tensorflow==2.15.0
!pip install statsmodels==0.14.0
!pip install numpy==1.24.3
!pip install scikit-learn==1.3.0
!pip install seaborn==0.12.2
!pip install matplotlib==3.7.1
!pip install pandas==2.0.3
!pip install yfinance==0.2.20
!pip install pandas_datareader==0.10.0
!pip install pandas_ta==0.3.14b
!pip install projectpro


### **Import Libraries**

In [ ]:
# Import all necessary libraries
from datetime import datetime, timedelta
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import yfinance as yf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import MinMaxScaler
from pandas_datareader.data import DataReader
from pandas_datareader import data as pdr
import pandas_ta as ta
from projectpro import model_snapshot, checkpoint
from keras.layers import LSTM, SimpleRNN, Dropout
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

# Set configurations
keras.backend.set_image_data_format("channels_last")
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight") 
%matplotlib inline

# Override Yahoo Finance downloader
yf.pdr_override()

# Create output directories
os.makedirs('output', exist_ok=True)
os.makedirs('output/models', exist_ok=True)
os.makedirs('output/figures', exist_ok=True)

print("✓ All libraries imported successfully!")
print("✓ Output directories created!")


# Simple Neural Network demonstration with Abalone dataset

In [ ]:
# Simple Neural Network demonstration with Abalone dataset
print("Demonstrating neural network basics with Abalone dataset...")

# Define abalone features
abalone_features = ["Length", "Diameter", "Height", "Whole weight", "Shucked weight", 
                    "Viscera weight", "Shell weight", "Age"]

# Load abalone data
abalone_train = pd.read_csv(
    "https://storage.googleapis.com/download.tensorflow.org/data/abalone_train.csv",
    names=abalone_features)

abalone_test = pd.read_csv(
    "https://storage.googleapis.com/download.tensorflow.org/data/abalone_test.csv",
    names=abalone_features)

print("Abalone dataset loaded successfully!")
abalone_train.head()


In [ ]:
# Prepare abalone data
abalone_train_features = abalone_train.copy()
abalone_train_labels = abalone_train_features.pop('Age')

abalone_test_features = abalone_test.copy()
abalone_test_labels = abalone_test_features.pop('Age')

# Create normalization layer
normalize = tf.keras.layers.Normalization()
normalize.adapt(abalone_train_features)

# Create and compile abalone model
abalone_model = tf.keras.Sequential([
    normalize,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

abalone_model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# Train the model
print("Training Abalone model...")
abalone_model.fit(
    abalone_train_features, abalone_train_labels,
    epochs=10, validation_split=0.2, verbose=1
)

# Evaluate the model
predictions = abalone_model.predict(abalone_test_features)
squared_error = tf.keras.metrics.MeanSquaredError()
squared_error.update_state(abalone_test_labels, predictions)
print(f'Mean Squared Error for test set: {squared_error.result().numpy():.4f}')


In [ ]:
# Load Tesla stock data
print("Loading Tesla (TSLA) stock data...")

# Retrieve historical stock price data for TSLA
dataset = pdr.get_data_yahoo('TSLA', start='2012-01-01', end=datetime.now())

print(f"✓ Tesla stock data loaded successfully!")
print(f"Data shape: {dataset.shape}")
print(f"Date range: {dataset.index[0]} to {dataset.index[-1]}")

# Display first few rows
dataset.head()


In [ ]:
# Data quality check and preprocessing
print("Performing data quality checks...")

# Check for missing values
print("Missing values per column:")
print(dataset.isnull().sum())

# Fill missing values if any
dataset.fillna(method='ffill', inplace=True)

# Check for remaining missing values
if dataset.isnull().sum().sum() > 0:
    dataset.dropna(inplace=True)
    print("Remaining missing values removed")

print(f"✓ Final dataset shape: {dataset.shape}")
print("✓ Data preprocessing completed!")


In [ ]:
# Define time periods for Tesla (adjusted for Tesla's IPO and market stability)
tstart = 2017  # Start year for training
tend = 2022    # End year for training

print(f"Training period: {tstart} to {tend}")
print(f"Test period: {tend+1} onwards")

# Define plotting function
def train_test_plot(dataset, tstart, tend, stock_name="Tesla"):
    """Plot training and test data for stock prices"""
    plt.figure(figsize=(16, 6))
    
    # Plot training data
    dataset.loc[f"{tstart}":f"{tend}", "High"].plot(
        legend=True, label=f"Train (Before {tend+1})")
    
    # Plot test data
    dataset.loc[f"{tend+1}":, "High"].plot(
        legend=True, label=f"Test ({tend+1} and beyond)")
    
    plt.legend()
    plt.title(f"{stock_name} Stock Price - Training vs Test Data")
    plt.xlabel("Date")
    plt.ylabel("Price ($)")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Plot the data split
train_test_plot(dataset, tstart, tend, "Tesla")


In [ ]:
# Define data splitting function
def train_test_split(dataset, tstart, tend, columns=['High']):
    """Split dataset into training and test sets based on time periods"""
    train = dataset.loc[f"{tstart}":f"{tend}", columns].values
    test = dataset.loc[f"{tend+1}":, columns].values
    
    print(f"Training data shape: {train.shape}")
    print(f"Test data shape: {test.shape}")
    
    return train, test

# Split the dataset
training_set, test_set = train_test_split(dataset, tstart, tend)

# Scale the data
sc = MinMaxScaler(feature_range=(0, 1))
training_set = training_set.reshape(-1, 1)
training_set_scaled = sc.fit_transform(training_set)

print(f"✓ Training set scaled successfully!")
print(f"Scaled training set shape: {training_set_scaled.shape}")


In [ ]:
# Create sequences for time series modeling
n_steps = 60  # Increased from 1 to 60 for better temporal pattern capture
features = 1

def split_sequence(sequence, n_steps):
    """Split a sequence into input and output sequences for time series forecasting"""
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Create training sequences
X_train, y_train = split_sequence(training_set_scaled, n_steps)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], features)

print(f"✓ X_train shape: {X_train.shape}")
print(f"✓ y_train shape: {y_train.shape}")


In [ ]:
# Define evaluation and plotting functions
def plot_predictions(test, predicted, title, save_path=None):
    """Plot real and predicted values for time series forecasting"""
    plt.figure(figsize=(16, 8))
    
    plt.plot(test, color="blue", label="Actual", linewidth=2)
    plt.plot(predicted, color="red", label="Predicted", linewidth=2, alpha=0.8)
    
    plt.title(f'{title}', fontsize=16, fontweight='bold')
    plt.xlabel("Time", fontsize=12)
    plt.ylabel("Stock Price ($)", fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    
    plt.tight_layout()
    plt.show()

def calculate_metrics(test, predicted):
    """Calculate comprehensive evaluation metrics"""
    rmse = np.sqrt(mean_squared_error(test, predicted))
    mae = mean_absolute_error(test, predicted)
    mape = np.mean(np.abs((test - predicted) / test)) * 100
    
    test_direction = np.diff(test.flatten()) > 0
    pred_direction = np.diff(predicted.flatten()) > 0
    directional_accuracy = np.mean(test_direction == pred_direction) * 100
    
    print("="*50)
    print("MODEL EVALUATION METRICS")
    print("="*50)
    print(f"Root Mean Squared Error (RMSE): ${rmse:.2f}")
    print(f"Mean Absolute Error (MAE): ${mae:.2f}")
    print(f"Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
    print(f"Directional Accuracy: {directional_accuracy:.2f}%")
    print("="*50)
    
    return {
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'Directional_Accuracy': directional_accuracy
    }

def plot_loss(history, title="Training Loss", save_path=None):
    """Plot training loss over epochs"""
    plt.figure(figsize=(12, 6))
    
    plt.plot(history.history['loss'], label='Training Loss', linewidth=2)
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
    
    plt.title(title, fontsize=16, fontweight='bold')
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Loss plot saved to {save_path}")
    
    plt.tight_layout()
    plt.show()

print("✓ Evaluation functions defined successfully!")


In [ ]:
# Create enhanced RNN model
print("Building Enhanced RNN Model...")

model_rnn = Sequential([
    SimpleRNN(units=50, return_sequences=True, input_shape=(n_steps, features)),
    Dropout(0.2),
    SimpleRNN(units=50, return_sequences=True),
    Dropout(0.2),
    SimpleRNN(units=50),
    Dropout(0.2),
    Dense(units=25, activation='relu'),
    Dense(units=1)
])

# Compile the model
model_rnn.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=['mae']
)

print("RNN Model Architecture:")
model_rnn.summary()


# Train RNN Model

In [ ]:
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.0001
)

# Train RNN model
print("Training RNN model...")
history_rnn = model_rnn.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Plot training history
plot_loss(history_rnn, "RNN Training History", "output/figures/rnn_training_loss.png")


# Evaluate RNN Model

In [ ]:
# Prepare test data and make predictions
inputs = sc.transform(test_set.reshape(-1, 1))
X_test, y_test = split_sequence(inputs, n_steps)
X_test = X_test.reshape(-1, n_steps, features)

print(f"Test data prepared - X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

# Make predictions
predicted_stock_price_rnn = model_rnn.predict(X_test)
predicted_stock_price_rnn = sc.inverse_transform(predicted_stock_price_rnn)
y_test_actual = sc.inverse_transform(y_test.reshape(-1, 1))

# Plot predictions
plot_predictions(y_test_actual, predicted_stock_price_rnn, 
                "Tesla Stock Price Prediction - RNN", 
                "output/figures/rnn_predictions.png")

# Calculate metrics
rnn_metrics = calculate_metrics(y_test_actual, predicted_stock_price_rnn)


# Generate Future Predictions with RNN

In [ ]:
# Define sequence generation function
def sequence_generation(dataset, sc, model, steps_future, n_steps):
    """Generate future stock price predictions using sequence generation"""
    high_dataset = dataset.iloc[len(dataset) - len(test_set) - n_steps:]["High"]
    high_dataset = sc.transform(high_dataset.values.reshape(-1, 1))
    inputs = high_dataset[-n_steps:]
    
    predictions = []
    current_input = inputs.copy()
    
    for i in range(steps_future):
        curr_pred = model.predict(current_input.reshape(1, n_steps, features), verbose=0)
        predictions.append(curr_pred[0, 0])
        current_input = np.append(current_input[1:], curr_pred, axis=0)
    
    predictions = np.array(predictions).reshape(-1, 1)
    return sc.inverse_transform(predictions)

# Generate future predictions
steps_in_future = 30
print("Generating future predictions with RNN...")
future_predictions_rnn = sequence_generation(dataset, sc, model_rnn, steps_in_future, n_steps)

# Plot future predictions
plot_predictions(y_test_actual[:steps_in_future], future_predictions_rnn, 
                "Tesla Stock Price - RNN Future Predictions", 
                "output/figures/rnn_future_predictions.png")

# Save RNN model
model_rnn.save("output/models/tesla_rnn_model.h5")
print("✓ RNN model saved successfully!")


# Build and Train Enhanced LSTM Model

In [ ]:
# Create enhanced LSTM model
print("Building Enhanced LSTM Model...")

model_lstm = Sequential([
    LSTM(units=50, return_sequences=True, input_shape=(n_steps, features)),
    Dropout(0.2),
    LSTM(units=50, return_sequences=True),
    Dropout(0.2),
    LSTM(units=50),
    Dropout(0.2),
    Dense(units=25, activation='relu'),
    Dense(units=1)
])

# Compile the model
model_lstm.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=['mae']
)

print("LSTM Model Architecture:")
model_lstm.summary()

# Train LSTM model
print("Training LSTM model...")
history_lstm = model_lstm.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Plot training history
plot_loss(history_lstm, "LSTM Training History", "output/figures/lstm_training_loss.png")


In [ ]:
# Make predictions with LSTM
predicted_stock_price_lstm = model_lstm.predict(X_test)
predicted_stock_price_lstm = sc.inverse_transform(predicted_stock_price_lstm)

# Plot LSTM predictions
plot_predictions(y_test_actual, predicted_stock_price_lstm, 
                "Tesla Stock Price Prediction - LSTM",
                "output/figures/lstm_predictions.png")

# Calculate LSTM metrics
lstm_metrics = calculate_metrics(y_test_actual, predicted_stock_price_lstm)

# Generate future predictions with LSTM
print("Generating future predictions with LSTM...")
future_predictions_lstm = sequence_generation(dataset, sc, model_lstm, steps_in_future, n_steps)

# Plot LSTM future predictions
plot_predictions(y_test_actual[:steps_in_future], future_predictions_lstm, 
                "Tesla Stock Price - LSTM Future Predictions",
                "output/figures/lstm_future_predictions.png")

# Save LSTM model
model_lstm.save("output/models/tesla_lstm_model.h5")
print("✓ LSTM model saved successfully!")


# Prepare Multivariate Data with Technical Indicators

In [ ]:
# Prepare multivariate data with technical indicators
print("Preparing multivariate data with technical indicators...")

mv_features = 8
multi_variate_df = dataset.copy()

# Calculate technical indicators
print("Calculating technical indicators for Tesla...")

# RSI
multi_variate_df['RSI'] = ta.rsi(multi_variate_df.Close, length=14)

# Multiple EMAs
multi_variate_df['EMA_12'] = ta.ema(multi_variate_df.Close, length=12)
multi_variate_df['EMA_26'] = ta.ema(multi_variate_df.Close, length=26)
multi_variate_df['EMA_50'] = ta.ema(multi_variate_df.Close, length=50)

# MACD
macd_data = ta.macd(multi_variate_df.Close)
multi_variate_df['MACD'] = macd_data['MACD_12_26_9']

# Bollinger Bands
bb_data = ta.bbands(multi_variate_df.Close, length=20)
multi_variate_df['BB_Upper'] = bb_data['BBU_20_2.0']
multi_variate_df['BB_Lower'] = bb_data['BBL_20_2.0']

# Volume indicator
multi_variate_df['Volume_SMA'] = ta.sma(multi_variate_df.Volume, length=20)

# Create target variable
multi_variate_df['Target'] = multi_variate_df['Adj Close'] - multi_variate_df['Open']
multi_variate_df['Target'] = multi_variate_df['Target'].shift(-1)

# Clean data
multi_variate_df.dropna(inplace=True)
multi_variate_df.drop(['Volume', 'Close'], axis=1, inplace=True)

print(f"✓ Multivariate dataset shape: {multi_variate_df.shape}")


# Visualize Technical Indicators

In [ ]:
# Plot technical indicators
plt.figure(figsize=(16, 8))

# Plot 1: Stock Price and RSI
plt.subplot(2, 1, 1)
multi_variate_df.loc[f"{tstart}":f"{tend}", 'High'].plot(
    title="Tesla Stock Price", color='blue')
plt.ylabel("Price ($)")

plt.subplot(2, 1, 2)
multi_variate_df.loc[f"{tstart}":f"{tend}", 'RSI'].plot(
    title="Tesla RSI", color='orange')
plt.axhline(y=70, color='r', linestyle='--', alpha=0.7, label='Overbought')
plt.axhline(y=30, color='g', linestyle='--', alpha=0.7, label='Oversold')
plt.ylabel("RSI")
plt.legend()

plt.tight_layout()
plt.savefig("output/figures/tesla_rsi_analysis.png", dpi=300, bbox_inches='tight')
plt.show()

# Plot 2: Moving Averages
plt.figure(figsize=(16, 6))
multi_variate_df.loc[f"{tstart}":f"{tend}", ['High', 'EMA_12', 'EMA_26', 'EMA_50']].plot(
    title="Tesla Stock Price with Moving Averages")
plt.ylabel("Price ($)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("output/figures/tesla_ema_analysis.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Prepare multivariate training data
feat_columns = ['Open', 'High', 'RSI', 'EMA_12', 'EMA_26', 'EMA_50', 'MACD', 'BB_Upper']
label_col = ['Target']

# Split multivariate data
mv_training_set, mv_test_set = train_test_split(
    multi_variate_df, tstart, tend, feat_columns + label_col)

# Extract features and labels
X_train_mv = mv_training_set[:, :-1]
y_train_mv = mv_training_set[:, -1]
X_test_mv = mv_test_set[:, :-1]
y_test_mv = mv_test_set[:, -1]

print(f"Multivariate training data shape: {X_train_mv.shape}")
print(f"Multivariate test data shape: {X_test_mv.shape}")

# Scale multivariate data
mv_sc = MinMaxScaler(feature_range=(0, 1))
X_train_mv_scaled = mv_sc.fit_transform(X_train_mv)
X_test_mv_scaled = mv_sc.transform(X_test_mv)

# Create sequences for multivariate data
def create_multivariate_sequences(X, y, n_steps):
    """Create sequences for multivariate time series data"""
    X_seq, y_seq = [], []
    for i in range(n_steps, len(X)):
        X_seq.append(X[i-n_steps:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

mv_n_steps = 30
X_train_mv_seq, y_train_mv_seq = create_multivariate_sequences(
    X_train_mv_scaled, y_train_mv, mv_n_steps)
X_test_mv_seq, y_test_mv_seq = create_multivariate_sequences(
    X_test_mv_scaled, y_test_mv, mv_n_steps)

print(f"✓ Multivariate sequences created:")
print(f"X_train_mv_seq: {X_train_mv_seq.shape}")
print(f"X_test_mv_seq: {X_test_mv_seq.shape}")


# Build and Train Multivariate LSTM Model

In [ ]:
# Create multivariate LSTM model
print("Building Multivariate LSTM Model...")

model_mv = Sequential([
    LSTM(units=100, return_sequences=True, input_shape=(mv_n_steps, mv_features)),
    Dropout(0.3),
    LSTM(units=100, return_sequences=True),
    Dropout(0.3),
    LSTM(units=50),
    Dropout(0.3),
    Dense(units=50, activation='relu'),
    Dropout(0.2),
    Dense(units=25, activation='relu'),
    Dense(units=1)
])

# Compile the model
model_mv.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="mse",
    metrics=['mae']
)

print("Multivariate LSTM Model Architecture:")
model_mv.summary()

# Train multivariate LSTM
print("Training Multivariate LSTM model...")
history_mv = model_mv.fit(
    X_train_mv_seq, y_train_mv_seq,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

# Plot training history
plot_loss(history_mv, "Multivariate LSTM Training History", 
          "output/figures/mv_lstm_training_loss.png")


In [ ]:
# Make predictions with multivariate LSTM
predictions_mv = model_mv.predict(X_test_mv_seq)

# Plot multivariate LSTM predictions
plot_predictions(y_test_mv_seq, predictions_mv, 
                "Tesla Stock Price - Multivariate LSTM",
                "output/figures/mv_lstm_predictions.png")

# Calculate multivariate LSTM metrics
mv_metrics = calculate_metrics(y_test_mv_seq.reshape(-1, 1), predictions_mv)

# Save multivariate LSTM model
model_mv.save("output/models/tesla_multivariate_lstm_model.h5")
print("✓ Multivariate LSTM model saved successfully!")


# Model Comparison and Final Results

In [ ]:
# Create comprehensive model comparison
print("\n" + "="*60)
print("TESLA STOCK PREDICTION - MODEL COMPARISON")
print("="*60)

# Create comparison DataFrame
models_comparison = pd.DataFrame({
    'Model': ['RNN', 'LSTM', 'Multivariate LSTM'],
    'RMSE': [rnn_metrics['RMSE'], lstm_metrics['RMSE'], mv_metrics['RMSE']],
    'MAE': [rnn_metrics['MAE'], lstm_metrics['MAE'], mv_metrics['MAE']],
    'MAPE': [rnn_metrics['MAPE'], lstm_metrics['MAPE'], mv_metrics['MAPE']],
    'Directional_Accuracy': [rnn_metrics['Directional_Accuracy'], 
                           lstm_metrics['Directional_Accuracy'], 
                           mv_metrics['Directional_Accuracy']]
})

print(models_comparison.to_string(index=False))
print("="*60)

# Save comparison results
models_comparison.to_csv("output/tesla_model_comparison.csv", index=False)
print("✓ Model comparison saved to output/tesla_model_comparison.csv")


# Visualize Model Comparison

In [ ]:
# Create model comparison visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# RMSE comparison
axes[0, 0].bar(models_comparison['Model'], models_comparison['RMSE'], 
               color=['blue', 'green', 'red'])
axes[0, 0].set_title('RMSE Comparison')
axes[0, 0].set_ylabel('RMSE ($)')

# MAE comparison
axes[0, 1].bar(models_comparison['Model'], models_comparison['MAE'], 
               color=['blue', 'green', 'red'])
axes[0, 1].set_title('MAE Comparison')
axes[0, 1].set_ylabel('MAE ($)')

# MAPE comparison
axes[1, 0].bar(models_comparison['Model'], models_comparison['MAPE'], 
               color=['blue', 'green', 'red'])
axes[1, 0].set_title('MAPE Comparison')
axes[1, 0].set_ylabel('MAPE (%)')

# Directional Accuracy comparison
axes[1, 1].bar(models_comparison['Model'], models_comparison['Directional_Accuracy'], 
               color=['blue', 'green', 'red'])
axes[1, 1].set_title('Directional Accuracy Comparison')
axes[1, 1].set_ylabel('Accuracy (%)')

plt.tight_layout()
plt.savefig("output/figures/tesla_model_comparison.png", dpi=300, bbox_inches='tight')
plt.show()

print("✓ Model comparison visualization completed!")


# Final Summary and Conclusion

# Final project summary
print("\n" + "="*80)
print("TESLA STOCK PREDICTION PROJECT - FINAL SUMMARY")
print("="*80)

print(f"""
📊 PROJECT COMPLETION SUMMARY:
✓ Successfully adapted stock prediction from Apple (AAPL) to Tesla (TSLA)
✓ Implemented 3 enhanced deep learning models:
  - Enhanced RNN with multiple layers and dropout
  - Enhanced LSTM with improved architecture  
  - Multivariate LSTM with 8 technical indicators
✓ Comprehensive evaluation with multiple metrics
✓ Professional visualizations and model comparisons
✓ All models saved for future use

📈 DATASET INFORMATION:
- Stock: Tesla (TSLA)
- Data Range: {dataset.index[0].date()} to {dataset.index[-1].date()}
- Training Period: {tstart} to {tend}
- Test Period: {tend+1} onwards
- Total Records: {dataset.shape[0]:,}

🎯 BEST PERFORMING MODEL:
Based on the comparison metrics, the model with the lowest RMSE is recommended for Tesla stock prediction.

📁 OUTPUT FILES GENERATED:
- Models: output/models/tesla_*_model.h5
- Figures: output/figures/*.png
- Comparison: output/tesla_model_comparison.csv

🚀 NEXT STEPS:
1. Use the trained models for real-time Tesla stock predictions
2. Experiment with additional technical indicators
3. Implement ensemble methods combining all models
4. Consider external factors like news sentiment analysis
""")

print("="*80)
print("🎉 TESLA STOCK PREDICTION PROJECT COMPLETED SUCCESSFULLY!")
print("="*80)
